In [1]:
import numpy as np
from IPython.display import display, Markdown #this is for the text in our code
from astropy.coordinates import SkyCoord #import astropy 
import astropy.units as u #import the units


# Data for LDN 1616

LDN is a globule that we can localize in Orion.
In our [discussion](https://github.com/will-henney/globule-seminario/issues/1) we found that the closer star thata can ionized LDN 1616 are Singma Ori and Eps Ori that have an ionized flux incident of $Q_0=7.59\times 10^{47}s^{-1}$ and $Q_0=6\times10^{48}s^{-1}$ respectively (to obtain this value for Eps Ori we extrapolate the data from [Martins](https://ui.adsabs.harvard.edu/abs/2005A%26A...436.1049M/abstract)). We focus only in Eps Ori because is an orde higher than Sigma Ori. 

Now to estimate the ionizing flux incident we need to find the angular distance usind the data in astropy and then we calculate the distance between the stars and LDN 1616.

In [2]:
from astropy.coordinates import SkyCoord #import astropy and numpy
import numpy as np
import astropy.units as u #import the units

#we obtain the coordinates from SkyCoord 
eps_ori = SkyCoord.from_name('eps ori')      #epsilon Ori coordinates
LDN_1616 = SkyCoord.from_name('LDN 1616')    #LDN 1616 coordinates

separation = eps_ori.separation(LDN_1616)   #we obtain the separation angular between eps Ori and LDN 1616
print('the separation angular is:')       
separation                                  #this separation is in degrees

the separation angular is:


<Angle 7.60726434 deg>

For the distance we supouse that LDN 1616 and Epsi Ori are at the same distance, and for the distance we use $400 pc$ that is the distance to Orion OB stars. 
Using geometry we have that de distance from Epsi Ori to LDN 1616 is $D=2d\sin(\theta/2)$ and we have

In [3]:
d=distance = 400*u.pc # distance to Orion OB stars
Distance = 2*distance*np.sin((separation).value*np.pi/(180*2)) #Distance
Distance

<Quantity 53.06972339 pc>

For the ionizing flux incident now we use $F=\frac{L}{4\pi D^2}$ using $Q_0$, and we have

In [4]:
Q0_eps = 6.0e48/(1*u.second)   #Q_0 for epsilon Ori (photons/s)
Q0_sig = 7.59E47/(1*u.second)  #Q_0 for sigma Ori (photons/s)

flux_io_eps = Q0_eps/(4*np.pi*Distance**2)
flux_io_sig = Q0_sig/(4*np.pi*Distance**2)

display(Markdown(r'The ionizing flux of $'r'\epsilon$ Ori is 'f'{flux_io_eps} and for $'r'\sigma$ Ori the ionizing flux is 'f'{flux_io_sig}'))

The ionizing flux of $\epsilon$ Ori is 1.6953045922016264e+44 1 / (pc2 s) and for $\sigma$ Ori the ionizing flux is 2.1445603091350573e+43 1 / (pc2 s)

Again, we note that the ionizng flux for sigma Ori is an order less than eps Ori.

Now, for the radius of LDN 1616 we can see in the [image](https://github.com/will-henney/globule-seminario/issues/1#issuecomment-1414493501) that the angular radius is .2° and again, using the same geometry with a discance of 384 pc ([Pliyali](https://ui.adsabs.harvard.edu/abs/2022MNRAS.513.2039S)) we have

In [5]:
angular_radius_LDN_1616 = 0.2*u.degree  #diameter of LDN 1616
distance_globule = 384*u.pc             #distance to LDN 1616
globule_radius = distance_globule*np.sin((angular_radius_LDN_1616.value)*np.pi/180)
print('The globule radius in pc is:',globule_radius)

The globule radius in pc is: 1.3404101434459132 pc


To calculate the density we considered two terms, 
the advection of new atoms through the ionization front,  $n_0u_0$, and the re-combinations in the ionized flow,
$\int n^2\alpha_\beta dr=n_0^2h\alpha_\beta$. So we use $F_0=n_0 u_0+n_0^2 h\alpha_\beta$ to compute the ionized density $n_0$, where $u_0$ is the gas velocity, we use $10 km/s$, $h$ is the effective thickness, we take $h=0.1r_0$ where $r_0$ is the globule radius.

Therefore to compute the ionized density we resolved our quadratic function $n_0^2(h\alpha_\beta)+n_0(u_0)-F_0=0$
and we take only the positive solution, so the solution is $$n_0=\frac{-u_0+\sqrt{u_0^2+4h\alpha_\beta F_0}}{2h\alpha_\beta}$$

In [6]:
#values that we considered
h = 0.1 * globule_radius.to('cm')           #The effective thickness
alpha_beta = 2.3E-13 * u.cm**3 / (u.second) #The recombination coefficient
u_0 = 10 * u.kilometer / (u.second)         #The gas velocity
cm_s = u.cm / u.second                      #convert the gas velocity to cgs
cms = (u.cm**2 *u.second)**(-1)             # units for cgs
F_0 = flux_io_eps.to(cms)                   #flux ionizing for epsilon Ori
#coefficients to solve the qudratic function
a = h*alpha_beta
b = u_0.to(cm_s)
c = -F_0
n_0 = (-b+(b**2-(4*a*c))**(1/2))/(2*a)
n_0

<Quantity 9.39983214 1 / cm3>

_Now what happened if we considered only one of the tow terms?_

_If we only take the advection term then the ionized density is_ $n_0=\frac{F_0}{u_0}$


_And similarly if we only take the re-combinations in the ionized flow then the ionized density is_ $n_0=\sqrt{\frac{F_0}{h\alpha_\beta}}$ _and we have_

In [7]:
adv_term = F_0/(u_0.to(cm_s))
rec_term = (F_0/(h*alpha_beta))**(1/2)
display(Markdown(r'The density only cosidered the advection term is $'r'n_{0,adv}$ = 'f'{adv_term} and iif we only considere the re-combination term then $'r'n_{0,rec}$ = 'f'{rec_term}'))

The density only cosidered the advection term is $n_{0,adv}$ = 17.80519145831026 1 / cm3 and iif we only considere the re-combination term then $n_{0,rec}$ = 13.680919281702542 1 / cm3

## Alternative estimate of density from H$\alpha$ surface brightness

For the emission measure we have that $EM=n_o^2\ell$, therefore $$n_0=\sqrt{\frac{EM}{\ell}}$$ and using data of an [image](https://github.com/will-henney/globule-seminario/issues/3#issuecomment-1416062507) we have an Intensity of 30 in Rayleigh, so we need change to EM and we have

$$I=\int \frac{f_{H_\alpha}\alpha_\beta n_e n_p}{4\pi}dz=\frac{f_{H_\alpha}\alpha_\beta}{4\pi}EM_{cm^{-5}}=\frac{f_{H_\alpha}\alpha_\beta(1\frac{pc}{cm})}{4\pi}EM_{cm^{-5}pc}$$ using typical values we have $f_{H_\alpha}\alpha_\beta\approx 1.17\times10^{-13}$ therefore
$$\frac{I}{cgs}=\frac{1.17\times10^{-13}3.086\times10^{18}}{4\pi}EM=\frac{3.61\times10^{5}}{4\pi}EM$$
and in Rayleighs we have that $$1 Ry=\frac{10^6}{4\pi} photons/s/cm^2/str$$
in units of cgs, finally we have that $$\frac{I}{Ry}=\frac{4\pi}{10^6}\frac{3.61\times10^5}{4\pi}EM=0.36EM$$
$$\Rightarrow EM=2.77\frac{I}{Ry}.$$

For $\ell$ we use $$\ell=\Big(\frac{2}{\sqrt{10}}\Big)r_o,$$ tehrefore

In [8]:
n0_LDN=np.sqrt((2.77*30)/(2/np.sqrt(10)))
print('the density that we obser is', n0_LDN, 'cm^{-3}')

the density that we obser is 11.462662726434733 cm^{-3}


If we incluide the reconbinations in the shocked shell we have 

$F_0=n_0 u_0+n_0^2 h\alpha_\beta+n_0^2h_1\alpha_\beta$

and taking the density from $H_\alpha$ surface brightness we have

In [9]:
n_0 = n0_LDN * 1/(u.cm**3)
h1 = 0.12*globule_radius.to('cm')
F_0 = n_0*u_0.to(cm_s) + n_0**2*h*alpha_beta + n_0**2*h1*alpha_beta
F_0

<Quantity 38961212.83935183 1 / (cm2 s)>